In [19]:
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2 # for reading pdf files

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# nltk.download('wordnet')
import PyPDF2
from docx import Document

In [20]:
def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    #stemmer = nltk.PorterStemmer()
    # Convert text to lowercase
    tokens = nltk.word_tokenize(text.lower())  
    # Remove stopwords and lemmatize
    stopwords = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stopwords]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Remove punctuation
    # stemmed_tokens = [nltk.PorterStemmer().stem(token) for token in tokens]
    lemmatized_tokens_no_punct = [''.join(c for c in token if c not in string.punctuation) for token in lemmatized_tokens]
    stemmed_tokens_no_punct = [token for token in lemmatized_tokens_no_punct if token]
    return ' '.join(lemmatized_tokens_no_punct)

In [21]:
preprocess("Hello, world! This is a test sentence. It contains punctuation, such as commas, periods, exclamation marks, and question marks. Does the function work correctly?")

'hello world test sentence contains punctuation comma period exclamation mark question mark function work correctly'

In [22]:
def extract_keywords(text, n):
    preprocessed_text = preprocess(text)
    
    # I can use pre trained model like BERT, GPT-2, etc. for keyword extraction here later 
    vectorizer = TfidfVectorizer() #ngram_range=(1, 2) for bigrams 
    # Fit and transform the preprocessed text
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text])
    
    feature_names = vectorizer.get_feature_names_out()
    #Get the TF-IDF scores for each feature
    tfidf_scores = tfidf_matrix.toarray()[0]#.flatten()
    # Create a dictionary of feature names and their corresponding TF-IDF scores
    tfidf_dict = dict(zip(feature_names, tfidf_scores))
    #sort the dictionary by TF-IDF scores in descending order
    sorted_tfidf_dict = dict(sorted(tfidf_dict.items(), key=lambda item: item[1], reverse=True))
    keywords = list(sorted_tfidf_dict.keys())[:n]
    return keywords

In [23]:
extract_keywords("Anime is getting very popular in India. The rise of Anime got at a very high volume when the COVID virus hit and the lockdown started. Though, I have been watching it since 2013 with my first favourite anime being Attack on Titan. Attack on Titan was first aired on 2013 and it was massive hit as an anime and the community really like it.", n=5)

['anime', '2013', 'attack', 'first', 'hit']

In [24]:
def read_file(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as file:
            text = file.read()
    elif file_path.endswith('.pdf'):
        pdf_file_obj = open(file_path, 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
        text = " ".join(page.extract_text() for page in pdf_reader.pages)
        pdf_file_obj.close()
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        text = " ".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        print("Unsupported file type")
        text = ""
    return text

In [25]:
file_path = 'C:/Users/Naman/Downloads/Anime.txt'  
text = read_file(file_path)
keywords = extract_keywords(text, n=5)
print(keywords)

['anime', '2013', 'attack', 'first', 'hit']


In [26]:
file_path = 'C:/Users/Naman/Documents/Naman_Singh_Resume.pdf'  
text = read_file(file_path)
keywords = extract_keywords(text,5)
print(keywords)

['2023', 'github', 'learning', 'model', 'accuracy']


In [27]:
file_path = 'C:/Users/Naman/Downloads/Final_project_report.docx'  
text = read_file(file_path)
keywords = extract_keywords(text,5)
print(keywords)


['summarization', 'user', 'system', 'using', 'knee']
